In [19]:
%cd /hpc2hdd/home/xzou428/Yuhao/HiGPT-tune-lightning/

import json
import os.path as osp
import os
import torch as th
import re
import pandas as pd
from tqdm import tqdm 
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import label_binarize
import numpy as np
from pathlib import Path
from itertools import combinations


res_dict = {}
for file in Path("checkpoints/higpt-stage2-llama3-IMDB_Few_Shot-epoch30-8192-full_finetune/lightning_logs/version_2/predict_eval_ACM_1000_20240802_19").glob("*.txt"):
    res_dict[file.stem.strip("[]").strip("'")] = file.read_text().strip()

# for file in Path("checkpoints/higpt-stage2-llama3-IMDB_Few_Shot-epoch30-8192-full_finetune/lightning_logs/version_1/predict_20240802_152618").glob("*.txt"):
#     res_dict[file.stem.strip("[]").strip("'")] = file.read_text().strip()
# assert len(res_dict) == 999

ground_truth = json.load(open("dataset/ACM_test_1000/test_cot_prompts_higpt_with_skg.json"))
ground_truth = {item['id']: item for item in ground_truth}

class_map = {'database': 0, 'wireless communication': 1, 'data mining': 2}

pattern = r'(Database|Wireless Communication|Data Mining)'


correct = 0
total = len(res_dict)

trues = []
preds = []

for nid in res_dict: 
    gpt_res = res_dict[nid]
    truth_item = ground_truth[nid]

    matches = re.search(pattern, gpt_res, re.I)
    # matched_label = matches.group(1) if matches else 'database'
    if matches: 
        matched_label = matches.group(1)
    else: 
        matched_label = 'Database'
    matched_label = matched_label.lower()

    true_y = truth_item['graph']['label']

    pred_y = class_map[matched_label]

    trues.append(true_y)
    preds.append(pred_y)

    if true_y == pred_y:
        correct = correct + 1
        # print("Correct:", nid, matched_label, gpt_res, true_y)
    else:
        # print("Incorrect:", nid, matched_label, gpt_res, true_y)
        pass

acc = correct / total

print(total)
print("Accuracy:", acc)

num_classes = 3
confusion_matrix = np.zeros((num_classes, num_classes))

for true_label, predicted_label in zip(trues, preds):
    confusion_matrix[true_label, predicted_label] += 1

# Accuracy
accuracy = np.trace(confusion_matrix) / np.sum(confusion_matrix)

# F1-score and Macro-F1
precision = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=1)
recall = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=0)
f1_score = 2 * precision * recall / (precision + recall)
macro_f1 = np.mean(f1_score)

micro_precision = np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)
micro_recall = np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)
micro_f1 = 2 * micro_precision * micro_recall / (micro_precision + micro_recall)

print("Accuracy:", accuracy)
print("F1 score:", f1_score.mean())
print("Macro F1:", macro_f1)
print("Micro F1:", micro_f1)

print(classification_report(trues, preds, target_names=list(class_map.keys()), digits=5))
# print("AUC-ROC:", roc_auc_score(trues, preds, multi_class='ovo', average='weighted'))

# Binarize the labels and predictions
labels_binarized = label_binarize(trues, classes=[0, 1, 2])
preds_binarized = label_binarize(preds, classes=[0, 1, 2])

# Calculate the AUC score for each class
auc_scores = []
for i in range(num_classes):
    auc = roc_auc_score(labels_binarized[:, i], preds_binarized[:, i])
    auc_scores.append(auc)

# Calculate the average AUC score
average_auc = np.mean(auc_scores)

print(f"AUC score for each class: {auc_scores}")
print(f"Average AUC OvR score: {average_auc}")


# # Initialize a list to store AUC scores
# auc_scores = []
# labels = trues
# # Iterate through all pairs of classes
# for (class1, class2) in combinations(range(num_classes), 2):
#     # Extract indices for the current pair
#     idx = np.where((labels == class1) | (labels == class2))[0]
    
#     # Ensure both classes are present
#     if len(idx) == 0 or len(np.unique(labels[idx])) < 2:
#         continue
    
#     # Filter predictions and labels for the current pair
#     preds_pair = preds[idx]
#     labels_pair = labels[idx]
    
#     # Binarize labels for the current pair
#     labels_pair = np.where(labels_pair == class1, 0, 1)
#     preds_pair = np.where(preds_pair == class1, 0, 1)
    
#     # Calculate the AUC score for the current pair
#     auc = roc_auc_score(labels_pair, preds_pair)
#     auc_scores.append(auc)

# # Calculate the average AUC score
# if auc_scores:  # Check if auc_scores is not empty
#     average_auc = np.mean(auc_scores)
# else:
#     average_auc = np.nan  # or any other value indicating no valid pairs were found

# print(f"AUC score for each pair: {auc_scores}")
# print(f"Average AUC OvO score: {average_auc}")

/hpc2hdd/home/xzou428/Yuhao/HiGPT-tune-lightning
999
Accuracy: 0.6256256256256256
Accuracy: 0.6256256256256256
F1 score: 0.6056783552985715
Macro F1: 0.6056783552985715
Micro F1: 0.6256256256256256
                        precision    recall  f1-score   support

              database    0.52900   0.70154   0.60317       325
wireless communication    0.98182   0.33645   0.50116       321
           data mining    0.63100   0.81870   0.71270       353

              accuracy                        0.62563       999
             macro avg    0.71394   0.61889   0.60568       999
          weighted avg    0.71054   0.62563   0.60910       999

AUC score for each class: [0.7001757589591417, 0.6667493728117332, 0.7785434883659741]
Average AUC OvR score: 0.7151562067122829
AUC score for each pair: []
Average AUC OvO score: nan


/hpc2hdd/home/xzou428/miniconda3/envs/yuh/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
/tmp/ipykernel_2309458/2099491970.py:120: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  idx = np.where((labels == class1) | (labels == class2))[0]


In [50]:
%cd /hpc2hdd/home/xzou428/Yuhao/HiGPT-tune-lightning/

import json
import os.path as osp
import os
import torch as th
import re
import pandas as pd
from tqdm import tqdm 
from sklearn.metrics import classification_report
import numpy as np
from pathlib import Path


res_dict = {}
for file in Path("checkpoints/higpt-stage2-llama3-MIX-epoch30-8192-full_finetune/lightning_logs/version_2/predict_eval_ACM_1000_20240802_162401").glob("*.txt"):
    res_dict[file.stem.strip("[]").strip("'")] = file.read_text().strip()

for file in Path("checkpoints/higpt-stage2-llama3-MIX-epoch30-8192-full_finetune/lightning_logs/version_2/predict_eval_ACM_1000_20240802_162853").glob("*.txt"):
    res_dict[file.stem.strip("[]").strip("'")] = file.read_text().strip()
# assert len(res_dict) == 999

ground_truth = json.load(open("dataset/ACM_test_1000/test_cot_prompts_higpt_with_skg.json"))
ground_truth = {item['id']: item for item in ground_truth}

class_map = {'database': 0, 'wireless communication': 1, 'data mining': 2}

pattern = r'(Database|Wireless Communication|Data Mining)'


correct = 0
total = len(res_dict)

trues = []
preds = []

for nid in res_dict: 
    gpt_res = res_dict[nid]
    truth_item = ground_truth[nid]

    matches = re.search(pattern, gpt_res, re.I)
    # matched_label = matches.group(1) if matches else 'database'
    if matches: 
        matched_label = matches.group(1)
    else: 
        matched_label = 'Database'
    matched_label = matched_label.lower()

    true_y = truth_item['graph']['label']

    pred_y = class_map[matched_label]

    trues.append(true_y)
    preds.append(pred_y)

    if true_y == pred_y:
        correct = correct + 1
        # print("Correct:", nid, matched_label, gpt_res, true_y)
    else:
        # print("Incorrect:", nid, matched_label, gpt_res, true_y)
        pass

acc = correct / total

print(total)
print("Accuracy:", acc)

num_classes = 3
confusion_matrix = np.zeros((num_classes, num_classes))

for true_label, predicted_label in zip(trues, preds):
    confusion_matrix[true_label, predicted_label] += 1

# Accuracy
accuracy = np.trace(confusion_matrix) / np.sum(confusion_matrix)

# F1-score and Macro-F1
precision = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=1)
recall = np.diag(confusion_matrix) / np.sum(confusion_matrix, axis=0)
f1_score = 2 * precision * recall / (precision + recall)
macro_f1 = np.mean(f1_score)

micro_precision = np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)
micro_recall = np.sum(np.diag(confusion_matrix)) / np.sum(confusion_matrix)
micro_f1 = 2 * micro_precision * micro_recall / (micro_precision + micro_recall)

print("Accuracy:", accuracy)
print("F1 score:", f1_score.mean())
print("Macro F1:", macro_f1)
print("Micro F1:", micro_f1)

/hpc2hdd/home/xzou428/Yuhao/HiGPT-tune-lightning
942
Accuracy: 0.5583864118895966
Accuracy: 0.5583864118895966
F1 score: 0.5450397147666147
Macro F1: 0.5450397147666147
Micro F1: 0.5583864118895966
